# Import Dependencies

In [ ]:
import numpy as np  # Numerical Analysis
import os # operating Systems
from PIL import Image, ImageTk
from io import BytesIO

In [ ]:
import requests
from tkinter import Tk, Label, Canvas, Entry, Button, NW

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D#Input, Hidden, Output Node
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image as img_utils

# CNN Network

In [ ]:
model = Sequential()
#Input Layer
model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# 1st Hidden Layer
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Hidden Layer
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Bridge Layer
model.add(Flatten())

#Output Layer
model.add(Dense(128, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

# compile

In [ ]:
model.compile(optimizer='adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
train_datagenerator = ImageDataGenerator(rescale = 1.0/255.0,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)

In [ ]:
test_datagenerator = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
training_datasets = train_datagenerator.flow_from_directory("dataset/training_set/", target_size= (128, 128), batch_size = 32, class_mode = "categorical")
testing_datasets = test_datagenerator.flow_from_directory("dataset/test_set/", target_size= (128, 128), batch_size = 32, class_mode = "categorical")

In [ ]:
model.fit(training_datasets, steps_per_epoch= 801/32, epochs = 10, validation_steps=200/32)

In [ ]:
model.save("Models/models.h5")
model.save_weights("Models/models_weight.h5")

In [ ]:
image_height, image_weight = 128, 128
models_locations = "Models/models.h5"
models_weight_locations = "Models/models_weight.h5"

final_model = load_model(models_locations)
final_model.load_weights(models_weight_locations)

In [ ]:
url = '' # global Variable
window = Tk()
window.title("Image Classification With CNN")
window.geometry("800x800")
label = Label(window, text = " Please Enter Your Coustom Image URL with the same Training image dot extension", font = ("Halvetica", 16))
label.pack()

def Enter():
    global url
    label.configure()
    url  = (User_input.get())
    print(" Here is your input url:\n", url)
    
    response = requests.get(url)
    test_image = Image.open(BytesIO(response.content))
    put_image = test_image.resize((80, 80))
    test_image = test_image.resize((128, 128))
    
    img = ImageTk.PhotoImage(put_image)
    pic = Label(image = img)
    pic.pack()
    
    pic.image = img
    test_image = img_utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    
    
    results = model.predict_on_batch(test_image)
    
    if results[0][0] == 1:
        res = "French Fries"
    elif results[0][1] == 1:
        res = "Fizza"
    elif results[0][2] == 1:
        res = "Samossa"
    else:
        res = "Error"
        
    output = Label(window, text = "Predicted Class: " + res, font = ("Halvetica", 16))
    output.pack()

# input and predict

In [ ]:
User_input = Entry()
User_input.pack()
button = Button(window, text = "Detected Class :", font = ("Halvetica", 16), command = Enter)
button.pack()
window.mainloop()